In [ ]:
import picos as pic
import cvxopt as cvx

import numpy as np
from numpy import linalg as LA
from scipy.special import comb as nchoosek # n Choose k (n ueber k)

In [ ]:
### Seite 37 (im Text zwischen (4.12) und (4.13))
def _N(n):
    return np.diag([p for p in xrange(-n, 0, 1)])

### Seite 55; (4.64)
def _M(n):
    return np.diag([p for p in xrange(0, n, 1)])

### Seite 55; (4.64)
def _P(l, k, n):
    I = np.eye(n)
    Mn = _M(n)
    P = I
    if k == 0:
        pass # P=I
    else:
        for q in xrange(0, k, 1):
            P = P * ((l-q)*I + Mn)
    return cvx.matrix(P)

In [ ]:
##############################
# Boris Diss Reactor         #
##############################
A = np.matrix([[1, 0],
              [0, -0.5]])
a = -A[-1][:].T ### !!!!!
b = np.matrix([[1],[-0.5]])
c = np.matrix([1, 0])
u_max = 1
n = 2

X0 = [np.matrix([-0.9, -2.8]).T,
      np.matrix([-0.9, 2.8]).T,
      np.matrix([0.9, -2.8]).T,
      np.matrix([0.9, 2.8]).T]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c

N = cvx.matrix(_N(n))
M = cvx.matrix(_M(n))

##### Entwurf parameter #####
beta = 2 # beta >=1 !

In [ ]:
# S. 78 Boris (LMI-Entwurf)
def convex_func(gamma_val, mu_val=1, verbose=0):
    ##############################
    # Convex Problem (35)        #
    ##############################
    prob = pic.Problem()

    # Constants
    AA = pic.new_param('A', A)
    II = pic.new_param('I_n', np.eye(n))
    III = pic.new_param('I_n+1', np.eye(n+1))
    aa = pic.new_param('a', a)
    bb = pic.new_param('b', b)
    XX0 = pic.new_param('X0', X0)

    #print "mu_inserted", mu_val
    mu = pic.new_param('mu', mu_val**2)

    ## REMARK THIS!!!! gamma is optimization variable but not convex, thus to be bisected to find "bigg-ish" value
    gamma = pic.new_param('gamma', gamma_val)

    # Problem
    prob = pic.Problem()

    # Parameters
    QQ = prob.add_variable('Q', (n, n), vtype='symmetric')
    zz = prob.add_variable('z', n)
    zz_star = prob.add_variable('z_star', n)

    # Objective
    prob.set_objective('find', None)

    # Constraints
    prob.add_constraint(QQ >> 0)
    
    #(A.10)
    prob.add_list_of_constraints([((QQ          & XX0[i]) //
                                   (XX0[i].T     & 1      )) >> 0
                                       for i in range(0, len(X0))])
    #(A.11)
    prob.add_constraint(((QQ   & zz) //
                         (zz.T & 1)) >> 0)

    #(A.12)
    prob.add_constraint(((QQ        & zz_star) //
                         (zz_star.T & mu)) >> 0)
    
    #(A.13)
    prob.add_constraint(QQ*AA + AA.T*QQ - bb*zz.T - zz*bb.T << 0) # STRICT SMALLER THAN!
    
    #(A.14)
    prob.add_constraint(QQ*AA + AA.T*QQ - bb*zz_star.T - zz_star*bb.T << -2*(gamma*QQ)) # STRICT SMALLER THAN!
    
    prob.solve(verbose=verbose, solver='cvxopt')
    return prob

print convex_func(1, 1, 1) 

In [ ]:
## Lets bisect
# Expects min_val to be valid, and max_val to be not valid
# func only taking on (scalar) argument -> gamma
def bisect_prob(min_val, max_val, func, diff=1e-5, max_iter=50, _iteration=0):
    if _iteration > max_iter or (max_val - min_val)/2.0 <= diff:
        print "Bisect done:", min_val
        print "Iter:", _iteration, ", Diff:", (max_val - min_val)/2.0
        return min_val
    else:
        mid_val = min_val+(max_val - min_val)/2.0
        #print "1. Evaluating: ", mid_val
        try:
            prob = func(mid_val)
            Q = prob.get_valued_variable('Q')
            z = prob.get_valued_variable('z')
            z_star = prob.get_valued_variable('z_star')
        except Exception as e:
            #print "Problem not solved!"
            #print e
            max_val = mid_val
        else:
            #print "Problem solved!"
            #print Q, z, z_star
            min_val = mid_val
        finally:
            #print "2. Recursing: ", min_val, max_val
            return bisect_prob(min_val, max_val, func, diff, max_iter,_iteration+1)
        #return min_val, max_val
    

In [ ]:
#%%timeit ~4sec
val = bisect_prob(0, 1, convex_func)
print val
print convex_func(val).get_valued_variable('Q')

In [ ]:
## Seite 47 Boris
# Wähle p_min, mu_start, zeta, zeta_start
def step1(p_min, mu_start, zeta, zeta_start):
    ####################################################
    # Löse (convex_func) mit mu = 1 -> l_star -> lambda_hat_i(1)=lambda_i(A-b*l_star.T)
    
    val = 0.11109
    #val = bisect_prob(0, 1, convex_func)
    
    prob = convex_func(val)
    Q = prob.get_valued_variable('Q')
    z_star = np.matrix(prob.get_valued_variable('z_star'))
#    print z_star
    l_star = LA.inv(Q)*z_star
#    print l_star
#    print A-b*l_star.T
    lambda_hat_1 = LA.eigvals(A-b*l_star.T)

    ####################################################
    ## Löse (convex_func) mit mu* >= 1 -> l_star -> lambda_hat_i_star(1)=lambda_i(A-b*l_star.T)
    mu_val = 1.1 ## TODO
    val = bisect_prob(0, 1, lambda g: convex_func(g, mu_val=mu_val))
    #val = bisect_prob(1, 10, lambda val_mu: bisect_prob(bisect_prob(0, 1, lambda g: convex_func(g, mu=val_mu)), val_mu))
    #gamma_max = bisect_prob(0, 10, convex_func)
    
    prob = convex_func(val, mu_val)
    Q = prob.get_valued_variable('Q')
    z_star = np.matrix(prob.get_valued_variable('z_star'))
#    print z_star
    l_star = LA.inv(Q)*z_star
#    print l_star
#    print A-b*l_star.T
    lambda_hat__star_1 = LA.eigvals(A-b*l_star.T)
    
step1(1.0/20, 1.5, 2.5, 5)

In [ ]:
l